In [2]:
import sys
import os
import json
import xml.etree.ElementTree as ET

# 第一步：为每一个文件名分配一个独立id

In [7]:
def get_filename_as_int(filename):
    idx = len(img_name_to_id)
    idx += 1
    img_name_to_id[filename]=idx

# xmllist.txt为所有的xml的文件名，如1_a_A.xml, ...
xmllist = r'C:/Users/LiangXuewei/Desktop/yinlie/xmllist.txt'


with open(xmllist, 'r') as f_xml:
    xmls = f_xml.readlines()

xmls = [i.strip().split('.')[0] for i in xmls]

img_name_to_id={}
for i in xmls:
    get_filename_as_int(i)

In [8]:
img_name_to_id['155205180110110203210001 3 3']

12

# 第二步：voc转coco

In [9]:
def get(root, name):
    vars = root.findall(name)
    return vars

def get_and_check(root, name, length):
    vars = root.findall(name)
    if len(vars) == 0:
        raise NotImplementedError('Can not find %s in %s.'%(name, root.tag))
    if length > 0 and len(vars) != length:
        raise NotImplementedError('The size of %s is supposed to be %d, but is %d.'%(name, length, len(vars)))
    if length == 1:
        vars = vars[0]
    return vars

def get_id(filename):
    return img_name_to_id[filename.split('.')[0]]

def convert(xml_list, xml_dir, json_file):
    list_fp = open(xml_list, 'r')
    json_dict = {"images":[], "type": "instances", "annotations": [],
                 "categories": []}
    categories = PRE_DEFINE_CATEGORIES
    bnd_id = START_BOUNDING_BOX_ID
    for line in list_fp:
        line = line.strip()
        print("Processing %s"%(line))
        xml_f = os.path.join(xml_dir, line)
        tree = ET.parse(xml_f)
        root = tree.getroot()
        path = get(root, 'path')
        if len(path) == 1:
            filename = os.path.basename(path[0].text)
        elif len(path) == 0:
            filename = get_and_check(root, 'filename', 1).text
        else:
            raise NotImplementedError('%d paths found in %s'%(len(path), line))
        ## The filename must be a number
        image_id = get_id(filename)
        size = get_and_check(root, 'size', 1)
        width = int(get_and_check(size, 'width', 1).text)
        height = int(get_and_check(size, 'height', 1).text)
        image = {'file_name': filename, 'height': height, 'width': width,
                 'id':image_id}
        json_dict['images'].append(image)
        ## Cruuently we do not support segmentation
        #  segmented = get_and_check(root, 'segmented', 1).text
        #  assert segmented == '0'
        for obj in get(root, 'object'):
            category = get_and_check(obj, 'name', 1).text
            if category not in categories:
                new_id = len(categories)
                categories[category] = new_id
            category_id = categories[category]
            bndbox = get_and_check(obj, 'bndbox', 1)
            xmin = int(get_and_check(bndbox, 'xmin', 1).text) - 1
            ymin = int(get_and_check(bndbox, 'ymin', 1).text) - 1
            xmax = int(get_and_check(bndbox, 'xmax', 1).text)
            ymax = int(get_and_check(bndbox, 'ymax', 1).text)
            assert(xmax > xmin)
            assert(ymax > ymin)
            o_width = abs(xmax - xmin)
            o_height = abs(ymax - ymin)
            ann = {'area': o_width*o_height, 'iscrowd': 0, 'image_id':
                   image_id, 'bbox':[xmin, ymin, o_width, o_height],
                   'category_id': category_id, 'id': bnd_id, 'ignore': 0,
                   'segmentation': []}
            json_dict['annotations'].append(ann)
            bnd_id = bnd_id + 1

    for cate, cid in categories.items():
        cat = {'supercategory': 'none', 'id': cid, 'name': cate}
        json_dict['categories'].append(cat)
    json_fp = open(json_file, 'w')
    json_str = json.dumps(json_dict)
    json_fp.write(json_str)
    json_fp.close()
    list_fp.close()


In [10]:
START_BOUNDING_BOX_ID = 1
PRE_DEFINE_CATEGORIES = {}
# 按需修改标签名称
# PRE_DEFINE_CATEGORIES = {"tuhen": 1, "qianao": 2, "zhezhou": 3, "tudian": 4,
#                           "qipao":5, "bianyuan": 6, "huahen": 7, "guahen": 8, "shenao": 9,
#                           "qingweiao": 10}
PRE_DEFINE_CATEGORIES = {"yinlie": 1, "jiaochayinlie": 2, "qita": 3}

# 转json
this_ = 'xmllist'  # 分别为train，val，test，trainval

xml_path = r'C:\Users\LiangXuewei\Desktop\yinlie/{}.txt'.format(this_)
ann_path = r'C:\Users\LiangXuewei\Desktop\yinlie\Annotations'  # 标签地址
out = 'voc_xxxx_{}.json'.format(this_)  # 默认格式为voc_xxxx_<this_>.json

# 执行voc2coco
convert(xml_path, ann_path, out)



Processing 154205180116110204560015 0 1.xml
Processing 154205180116110204560015 1 0.xml
Processing 154205180116110204560015 1 2.xml
Processing 154205180116110204560015 1 5.xml
Processing 154205180116110204560015 2 7.xml
Processing 154205180116110204560024 1 7.xml
Processing 154205180116110204560025 4 7.xml
Processing 154205180116110204560029 2 8.xml
Processing 154205180116110204560029 5 0.xml
Processing 154205180116110204560047 2 9.xml
Processing 154205180116110204560050 2 4.xml
Processing 155205180110110203210001 3 3.xml
Processing 15C205171229110200360472 0 2.xml
Processing 15C205171229110200361011 1 1.xml
Processing 15C205171229110200361196 0 8.xml
Processing 15C205171229110200361208 5 8.xml
Processing 15C205171229110200361209 4 5.xml
Processing 15C205171229110200361221 0 7.xml
Processing 15C205171229110200361230_1 3 7.xml
Processing 15C205171229110200361230 3 7.xml
Processing 15C205171229110200361232 5 5.xml
Processing 15C205171229110200361243 4 8.xml
Processing 15C2051712291102003

Processing 15C205180102110200922002 2 1.xml
Processing 15C205180102110200922018 4 2.xml
Processing 15C205180102110200922078 3 1.xml
Processing 15C205180102110200922078 4 3.xml
Processing 15C205180102110200922089 3 1.xml
Processing 15C205180103110201050189 1 0.xml
Processing 15C205180103110201050336 0 9.xml
Processing 15C205180103110201050364 1 3.xml
Processing 15C205180103110201050369 2 1.xml
Processing 15C205180103110201050371 3 3.xml
Processing 15C205180103110201050373 2 7.xml
Processing 15C205180103110201050377 2 2.xml
Processing 15C205180103110201050388 4 9.xml
Processing 15C205180103110201050391 2 1.xml
Processing 15C205180103110201052166 1 2.xml
Processing 15C205180103110201052166 5 1.xml
Processing 15C205180103110201052175 0 2.xml
Processing 15C205180103110201052175 2 1.xml
Processing 15C205180103110201052182 0 1.xml
Processing 15C205180103110201052185 4 4.xml
Processing 15C205180103110201052185 5 6.xml
Processing 15C205180103110201052186 2 7.xml
Processing 15C205180103110201052

Processing 15C205180107110202190800 4 2.xml
Processing 15C205180107110202190810 5 6.xml
Processing 15C205180107110202190898 1 8.xml
Processing 15C205180107110202191027 5 0.xml
Processing 15C205180107110202191294 1 2.xml
Processing 15C205180107110202191404 1 0.xml
Processing 15C205180107110202191404 1 4.xml
Processing 15C205180107110202191468 2 9.xml
Processing 15C205180107110202191477 2 7.xml
Processing 15C205180107110202191571 5 0.xml
Processing 15C205180107110202191615 3 0.xml
Processing 15C205180107110202191637 5 3.xml
Processing 15C205180107110202191637 5 9.xml
Processing 15C205180107110202191649 0 1.xml
Processing 15C205180107110202193437 5 3.xml
Processing 15C205180107110202193446 1 5.xml
Processing 15C205180107110202193459 4 9.xml
Processing 15C205180107110202193515 1 8.xml
Processing 15C205180107110202193516 1 8.xml
Processing 15C205180107110202193545 5 2.xml
Processing 15C205180107110202193548 5 3.xml
Processing 15C205180107110202193573 1 8.xml
Processing 15C205180107110202193

Processing 15C205180113110204020917 5 0.xml
Processing 15C205180113110204020953 2 2.xml
Processing 15C205180113110204020970 1 1.xml
Processing 15C205180113110204020983 0 9.xml
Processing 15C205180113110204020983 5 6.xml
Processing 15C205180113110204020993 0 4.xml
Processing 15C205180113110204020993 3 5.xml
Processing 15C205180113110204021003 0 6.xml
Processing 15C205180113110204021007 0 3.xml
Processing 15C205180113110204021007 2 7.xml
Processing 15C205180113110204021021 2 5.xml
Processing 15C205180113110204021042 0 1.xml
Processing 15C205180113110204021042 1 7.xml
Processing 15C205180113110204021042 5 6.xml
Processing 15C205180113110204021106 2 8.xml
Processing 15C205180113110204021137 5 9.xml
Processing 15C205180113110204021140 1 8.xml
Processing 15C205180113110204021153 1 7.xml
Processing 15C205180113110204021183 2 9.xml
Processing 15C205180113110204021183 3 0.xml
Processing 15C205180113110204021183 4 2.xml
Processing 15C205180113110204021183 5 0.xml
Processing 15C205180113110204021

# 第三步：根据生成的json文件，重写**&lt;this_ &gt;.txt**

In [11]:
# 根据json文件文件的id，重新写<this_>.txt
with open(xml_path, 'r') as f_x:
    s = f_x.readlines()

names = [i.strip().split('.')[0] for i in s]

train_dict = {}

for i in names:
    train_dict[i] = img_name_to_id[i]

# 对字典进行按值排序
f = zip(train_dict.keys(), train_dict.values())
ff = sorted(f)

out_list = []

for i in ff:
    out_list.append(i[0]+'\n')
# 保存为新的txt
with open('{}.txt'.format(this_), 'w') as f_w:
    f_w.writelines(out_list)